# Introduction to Cross-Validation - Lab

## Introduction

In this lab, you'll be able to practice your cross-validation skills!


## Objectives

You will be able to:

- Compare the results with normal holdout validation
- Apply 5-fold cross validation for regression

## Let's get started

This time, let's only include the variables that were previously selected using recursive feature elimination. We included the code to preprocess below.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.datasets import load_boston

boston = load_boston()

boston_features = pd.DataFrame(boston.data, columns = boston.feature_names)
b = boston_features["B"]
logdis = np.log(boston_features["DIS"])
loglstat = np.log(boston_features["LSTAT"])

# minmax scaling
boston_features["B"] = (b-min(b))/(max(b)-min(b))
boston_features["DIS"] = (logdis-min(logdis))/(max(logdis)-min(logdis))

#standardization
boston_features["LSTAT"] = (loglstat-np.mean(loglstat))/np.sqrt(np.var(loglstat))

In [2]:
X = boston_features[['CHAS', 'RM', 'DIS', 'B', 'LSTAT']]
y = pd.DataFrame(boston.target,columns = ['target'])

## Train test split

Perform a train-test-split with a test set of 0.20.

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [5]:
print(len(x_train), len(x_test), len(y_train), len(y_test))

404 102 404 102


Fit the model and apply the model to the make test set predictions

In [6]:
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()

In [7]:
result = linreg.fit(x_train, y_train)

In [8]:
y_hat_train = result.predict(x_train)
y_hat_test = result.predict(x_test)

Calculate the residuals and the mean squared error

In [9]:
from sklearn.metrics import mean_squared_error as mse

In [10]:
train_MSE = mse(y_train, y_hat_train)
test_MSE = mse(y_test, y_hat_test)

In [11]:
train_MSE

22.652349660324546

In [12]:
test_MSE

18.384562770971797

## Cross-Validation: let's build it from scratch!

### Create a cross-validation function

Write a function k-folds that splits a dataset into k evenly sized pieces.
If the full dataset is not divisible by k, make the first few folds one larger then later ones.

We want the folds to be a list of subsets of data!

In [33]:
def kfolds(data, k):
    # Force data as pandas dataframe
    # add 1 to fold size to account for leftovers
    data = pd.DataFrame(data)
    fold_size = len(data)//k
    remainder = len(data)%k
    folds = []
    start = 0
    for i in range(0,len(data), fold_size):
        if len(data) - i < fold_size:
            count = 0
            for j in range(start,len(data)):
                folds[count].append(data.iloc[start+count,:])
                count += 1
        else:
            folds.append(data[start:start + fold_size])
            start += fold_size
            
    return folds

### Apply it to the Boston Housing Data

In [34]:
# Make sure to concatenate the data again
len(bos_data)

506

In [35]:
bos_data = pd.concat([X.reset_index(drop=True), y], axis=1)

In [36]:
folds = kfolds(bos_data, 5)

### Perform a linear regression for each fold, and calculate the training and test error

Perform linear regression on each and calculate the training and test error.

In [56]:
test_errs = []
train_errs = []
k=5

for n in range(k):
    # Split in train and test for the fold
    train = pd.concat(fold for i, fold in enumerate(folds) if i != n)
    test = folds[n]
    # Fit a linear regression model
    model = linreg.fit(train[:,:-1], train[:,-1])
    #Evaluate Train and Test Errors

# print(train_errs)
# print(test_errs)

TypeError: unhashable type: 'slice'

In [57]:
folds[1][:-1]

,CHAS,RM,DIS,B,LSTAT,target
101,0.0,6.781,0.390808,0.996672,-0.555806,26.5
102,0.0,6.405,0.369415,0.177720,-0.012136,18.6
103,0.0,6.137,0.369415,0.993873,0.378596,19.3
104,0.0,6.167,0.321174,0.989384,0.235000,20.1
105,0.0,5.851,0.262627,0.992814,0.717270,19.5
106,0.0,5.836,0.282946,0.996898,0.925237,19.5
107,0.0,6.127,0.265716,0.976776,0.457274,20.4
108,0.0,6.474,0.323240,0.995814,0.226874,19.8
109,0.0,6.229,0.342236,0.985703,0.621518,19.4
110,0.0,6.195,0.379096,0.991401,0.323147,21.7


## Cross-Validation using Scikit-Learn

This was a bit of work! Now, let's perform 5-fold cross-validation to get the mean squared error through scikit-learn. Let's have a look at the five individual MSEs and explain what's going on.

Next, calculate the mean of the MSE over the 5 cross-validations and compare and contrast with the result from the train-test-split case.

##  Summary 

Congratulations! You now practiced your knowledge on k-fold crossvalidation!